In [1]:
diretorio_datasets = "/home/matteus-paula/Documents/dados/dados_tcc/remun_docentes"
#diretorio_datasets = "/home/caioabreu/workspace/tcc/datasets/investimento"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

> ## Dados Remuneração Docentes Muncipio:<br>
- https://www.fnde.gov.br/index.php/fnde_sistemas/siope/relatorios/arquivos-dados-analiticos

In [36]:
def read_docente(uf):
    df1_remun_docentes = pd.read_csv(f'{diretorio_datasets}/REMUNERACAO_PROFISSIONAIS_EDUCACAO_{uf}_2017.CSV', 
                                     delimiter=";", 
                                     encoding = "ISO-8859-1",
                                     usecols=["CO_MUNICIPIO", 
                                              "NO_PROFISSIONAL",
                                              "LOCAL_EXERCICIO",
                                              "CO_UF", 
                                              "VL_TOTAL", 
                                              "CATEG_PROFISSIONAL", 
                                              "TP_CATEGORIA"])
    list_categ = ['Docente habilitado em curso de pedagogia', 
                  'Docente habilitado em curso de licenciatura plena',
                  'Docente habilitado em curso de nível médio',
                  'Profissionais em efetivo exercício no âmbito da educação infantil e ensino fundamental.']

    if df1_remun_docentes.empty:
         return df1_remun_docentes
    else:
        df2_remun_docentes = df1_remun_docentes[df1_remun_docentes['CATEG_PROFISSIONAL'].isin(list_categ)
                                                 & (df1_remun_docentes['TP_CATEGORIA'] == 'PROFISSIONAIS DO MAGISTÉRIO')]
        df2_remun_docentes['VL_TOTAL'] = df2_remun_docentes['VL_TOTAL'].str.replace(',','.').astype('float')
        df4_remun_docentes = df2_remun_docentes[['CO_MUNICIPIO',
                                                 'NO_PROFISSIONAL',
                                                 'LOCAL_EXERCICIO',
                                                 'CO_UF', 
                                                 'VL_TOTAL']] \
                                               .groupby(['CO_MUNICIPIO',
                                                 'NO_PROFISSIONAL',
                                                 'LOCAL_EXERCICIO',        
                                                 'CO_UF',], as_index=False) \
                                               .agg(lambda x: round(x.sum(), 2))
        return df4_remun_docentes

- Soma do salario anual 

In [37]:
def media_rem_docente_uf(df):
    if df.empty:
        return df
    else:
        df1 = df[['CO_MUNICIPIO',
                  'CO_UF', 
                  'VL_TOTAL']] \
                .groupby(['CO_MUNICIPIO',
                          'CO_UF',], as_index=False) \
                .agg(lambda x: round(x.mean(),2))
        return df1

In [38]:
ufs = ['AC',
'AL',
'AP',
'AM',
'BA',
'CE',
'DF',
'ES',
'GO',
'MA',
'MT',
'MS',
'MG',
'PA',
'PB',
'PR',
'PE',
'PI',
'RJ',
'RN',
'RS',
'RO',
'RR',
'SC',
'SP',
'SE',
'TO']

COLUNAS = ['CO_MUNICIPIO',
           'CO_UF', 
           'VL_TOTAL']
df_rem_docentes = pd.DataFrame(columns=COLUNAS)
for uf in ufs:
    df = media_rem_docente_uf(read_docente(uf))
    if df.empty:
        print(uf)
    else:
        df_rem_docentes = df_rem_docentes.append(df)
        print(uf + ' - ' + str(df['CO_UF'].count()))

print(df_rem_docentes['CO_UF'].count())

/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


AC - 22
AL - 96
AP - 16
AM - 61
BA - 410
CE - 176
DF
ES - 78
GO - 217
MA - 204
MT - 140
MS - 78
MG - 844
PA - 133
PB - 222
PR - 391
PE - 180
PI - 217
RJ - 91
RN - 158
RS - 494
RO - 52
RR - 15
SC - 290
SP - 639
SE - 74
TO - 135
5433


IQR = 3 quartil - 1 quartil <br>
OUTLIER_BAIXO = 1 quartil - 1.5 IQR <br>
OUTLIER_CIMA = 3 quartil + 1.5 IQR

In [39]:
iqr = df_rem_docentes.VL_TOTAL.quantile(0.75) - df_rem_docentes.VL_TOTAL.quantile(0.25)

In [40]:
outlier_down = round(df_rem_docentes.VL_TOTAL.quantile(0.25) - (1.5 * iqr), 2)
outlier_up = round(df_rem_docentes.VL_TOTAL.quantile(0.75) + (1.5 * iqr), 2)

In [41]:
print(outlier_up)
print(outlier_down)

51295.17
-3572.59


In [42]:
df_rem_docentes.head()

,CO_MUNICIPIO,CO_UF,VL_TOTAL
0,120001,12,30339.05
1,120005,12,12965.51
2,120010,12,15376.11
3,120013,12,19510.97
4,120017,12,29418.67


In [43]:
df_rem_docentes_expurgo = df_rem_docentes[(df_rem_docentes.VL_TOTAL <= outlier_down) \
                                            | (df_rem_docentes.VL_TOTAL >= outlier_up)]
df_rem_docentes_expurgo['CO_UF'].count()

215

In [44]:
df_rem_docentes_normalizado = df_rem_docentes[(df_rem_docentes.VL_TOTAL > outlier_down) \
                                            & (df_rem_docentes.VL_TOTAL < outlier_up)]
df_rem_docentes_normalizado['CO_UF'].count()

5218

In [45]:
df_rem_docentes_normalizado['CO_MUNICIPIO_IBGE'] = df_rem_docentes_normalizado['CO_MUNICIPIO']
df_rem_docentes_normalizado = df_rem_docentes_normalizado[['CO_MUNICIPIO_IBGE', 'VL_TOTAL']]

/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_rem_docentes_expurgo.to_csv(f'{diretorio_datasets}/expurgo_remuneracao_media_docentes.csv', sep=';', encoding='utf-8', index=False)
df_rem_docentes_normalizado.to_csv(f'{diretorio_datasets}/remuneracao_media_docentes.csv', sep=';', encoding='utf-8', index=False)